<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

Import e instalação das bibliotecas necessárias

In [ ]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

     |████████████████████████████████| 174 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=886a89fb63bb6410726f954cbf686cbea04905ea4aa4faf8da21e3fe3d35b6ee
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [2]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [39]:
# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú0-9]+'

########## PATHS DE ARQUIVOS ##########

# Base após pré-processamento COM stopwords (Base 1)
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base após pré-processamento SEM stopwords (Base 2)
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Amostra Base 1
PATH_TESTE_BASE1 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base1_Com_StopWords.csv"  

# Amostra Base 2
PATH_TESTE_BASE2 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

# Lista de Palavrões
PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

In [43]:
#Carrega a base de dados com stopWords
#base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
base1 = pd.read_csv(PATH_TESTE_BASE1, index_col=0)
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito


In [5]:
#Carrega a base de dados sem stopWords
#base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
base2= pd.read_csv(PATH_TESTE_BASE2, index_col=0)
base2.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,famoso deus livre dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,ficando maluco weverton palmeiras convocado se...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,sambinha quarta hashtag bar seabra filho sexta...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...,2017 achei feio talvez meio diferente demorei ...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,chato gosto


In [6]:
base1.shape

(773, 6)

In [7]:
base2.shape

(773, 7)

Classificação Hate, Hate+ e outros

In [8]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

,hatelist
0,aberração
1,abestado
2,abestalhado
3,aborto
4,acabouamamata


In [41]:
# Carrega dicionário badword
dict_badword = pd.read_csv(PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR)
dict_badword.head(5)

,badword_list
0,babaca
1,boceta
2,boquete
3,bosseta
4,buceta


In [37]:
# Definição da função de classificação

def classifica(tweet, dicionario, LABEL1, LABEL2):
  # Tokeniza o tweet  
  tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
  tokens = tokenizer.tokenize(tweet)
  
  chave = dicionario.keys()[0]

  lista = list(dicionario[chave])
  lista_expressoes = []

  # Gera lista de expressões
  for termo in lista:
    termo_tokens = tokenizer.tokenize(termo)
    if(len(termo_tokens)>1): #expressoes
      lista_expressoes.append(termo)

  # Verifica se o tweet possui ao menos uma PALAVRA de hate
  for token in tokens:
    if token in lista:
      return LABEL1

  # Verifica se o tweet possui ao menos uma EXPRESSÃO de hate
  for palavra in lista_expressoes:
    result = re.subn(r'^' + palavra + '\s', palavra + ' ', tweet) # Substitui no início
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '$',' '+ palavra, tweet) # Substitui no final
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '\s', ' '+palavra+ ' ', tweet) #Substitui no meio
    if(result[1]):
      break

  if(result[1]):
    return LABEL1
  else:
    return LABEL2

In [44]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 1
base1['classificacao_hate'] = base1['text_limpo'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,classificacao_hate
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,not_hate
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,not_hate
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,not_hate
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...,hate
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,not_hate


In [19]:
# Salva a base1
base1.to_csv("base1_classificada.csv")

In [45]:
# Pega só tweets not_hate
base1_not_hate = base1[ base1['classificacao_hate']=='not_hate' ]


In [46]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base1_not_hate['classificacao_badword'] = base1_not_hate['text_limpo'].apply(lambda x: classifica(x,dict_badword,"badword","not badword"))
base1_not_hate.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,classificacao_hate,classificacao_badword
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,not_hate,not badword
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,not_hate,not badword
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,not_hate,not badword
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,not_hate,not badword
7,1347036901883211778,Thu Jan 07 04:26:38 +0000 2021,RT @BobjeffHD: Penso que Bolsonaro não deveria...,[],NaN,retwet penso que bolsonaro não deveria se mete...,not_hate,not badword


In [47]:
# Salva a base1
base1_not_hate.to_csv("base1_classificada2.csv")